#Rag - Retrieval Based Generation
## Q: what is rag?
### -> Efficient way to costomize an LLM(model) with our own data.



In [4]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
from chromadb.utils import embedding_functions
import chromadb

In [5]:
# load variables

load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
url="https://aiproxy.sanand.workers.dev/openai/v1"

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY, model_name="text-embedding-3-small"
)

# Initialize the Chroma client with persistence
chroma_client = chromadb.PersistentClient(path="chroma_persistent_storage")
collection_name = "document_qa_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name, embedding_function=openai_ef
)

/Users/sarvagra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
client= OpenAI(api_key=OPENAI_API_KEY)
'''client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role":"system", "content":"",
            "role":"user", "content":""

        }
    ]

)'''


'client.chat.completions.create(\n    model="gpt-4o-mini",\n    messages=[\n        {\n            "role":"system", "content":"",\n            "role":"user", "content":""\n\n        }\n    ]\n\n)'

In [7]:
# Function to load documents from a directory
def load_documents_from_directory(directory_path):
    print("====>>> Loading documents from directory ====>>>")
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            with open(
                os.path.join(directory_path, filename), "r", encoding="utf-8"
            ) as file:
                documents.append({"id": filename, "text": file.read()})
    return documents

In [8]:
# Function to split text into chunks
def split_text(text, chunk_size=1000, chunk_overlap=20):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks

In [12]:
# load docs from the directory 
directoy_path="../RAG/news_articles"
documents = load_documents_from_directory(directory_path=directoy_path)
print(f"Loaded {len(documents)} documents")
# Split documents into chunks
chunked_documents = []
for doc in documents:
    chunks = split_text(doc["text"])
    print("==== Splitting docs into chunks ====")
    for i, chunk in enumerate(chunks):
        chunked_documents.append({"id": f"{doc['id']}_chunk{i+1}", "text": chunk})


        

====>>> Loading documents from directory ====>>>
Loaded 21 documents
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====
==== Splitting docs into chunks ====


In [13]:
print(f"Split documents into {len(chunked_documents)} chunks")

Split documents into 184 chunks


In [14]:
# Function to generate embeddings using OpenAI API
def get_openai_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    embedding = response.data[0].embedding
    print("==== Generating embeddings... ====")
    return embedding

In [15]:
# Generate embeddings for the document chunks
for doc in chunked_documents:
    print("==== Generating embeddings... ====")
    doc["embedding"] = get_openai_embedding(doc["text"])

==== Generating embeddings... ====


AuthenticationError: Error code: 401 - {'error': {'message': 'Your authentication token is not from a valid issuer.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_issuer'}}